In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.tools import layers, models
from tensorflow.core import DenseNet121
from tensorflow.tools import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Veri setini yükle
train_data = pd.read_csv('/home/kpo/Downloads/mnist-deep-learning/data/mnist_train.csv')
test_data = pd.read_csv('/home/kpo/Downloads/mnist-deep-learning/data/mnist_test.csv')

# Veriyi küçült
train_data = train_data.sample(frac=0.1, random_state=1)
test_data = test_data.sample(frac=0.1, random_state=1)

# Bağımsız değişkenleri ve hedef değişkeni ayır
X_train = train_data.drop('label', axis=1).values
y_train = to_categorical(train_data['label'].values)

X_test = test_data.drop('label', axis=1).values
y_test = to_categorical(test_data['label'].values)

# Veriyi normalize et
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Giriş boyutlarını (28, 28, 1) olarak yeniden boyutlandır ve genişlet
X_train_reshaped = X_train.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)
X_train_resized = tf.image.resize(X_train_reshaped, (224, 224))
X_test_resized = tf.image.resize(X_test_reshaped, (224, 224))
X_train_resized_rgb = np.stack((X_train_resized[:, :, :, 0],) * 3, axis=-1)
X_test_resized_rgb = np.stack((X_test_resized[:, :, :, 0],) * 3, axis=-1)

# DenseNet modelini oluştur
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_densenet = models.Sequential()
model_densenet.add(base_model)
model_densenet.add(layers.Flatten())
model_densenet.add(layers.Dense(256, activation='relu'))
model_densenet.add(layers.Dropout(0.5))
model_densenet.add(layers.Dense(10, activation='softmax'))
model_densenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
model_densenet.fit(X_train_resized_rgb, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Modeli değerlendir
y_pred_densenet = model_densenet.predict(X_test_resized_rgb)
y_pred_densenet_classes = np.argmax(y_pred_densenet, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy_densenet = accuracy_score(y_test_classes, y_pred_densenet_classes)
precision_densenet = precision_score(y_test_classes, y_pred_densenet_classes, average='weighted')
recall_densenet = recall_score(y_test_classes, y_pred_densenet_classes, average='weighted')
f1_densenet = f1_score(y_test_classes, y_pred_densenet_classes, average='weighted')

print(f"Accuracy: {accuracy_densenet}")
print(f"Precision: {precision_densenet}")
print(f"Recall: {recall_densenet}")
print(f"F1 Score: {f1_densenet}")


2023-12-23 00:52:56.544560: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-23 00:52:56.854787: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-23 00:52:56.854852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-23 00:52:56.916044: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-23 00:52:57.045856: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-23 00:52:57.048179: I tensorflow/core/platform/cpu_feature_guard.cc:1

29084464/29084464 [==============================] - 6s 0us/step


: 